**План действий**
1. Применить маску к фотографиям \(убрать море, чтобы сеть быстрее обучалась\)
2. Использовать полноразмерные изображения \(выше точность\(?\)
3. SIFT / ROOTSIFT для извлечения точек
4. FAISS для сопоставления точек
5. Применить метод Keypoint Matching

![image](researchings/img.png 'Title')

Импорт библиотек

In [1]:
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import cv2
print('Библиотеки успешно загружены')

Библиотеки успешно загружены


Применить маску к изображениям

Переместить маски в зеркальную папку

In [80]:
dataset_directory = Path('Whale ReId 2_mm')
mask_directory = 'Whale ReId 2_mm masked'

for file in dataset_directory.glob('**/*.png'):
    Path(mask_directory, *file.parts[1:-1]).mkdir(parents=True, exist_ok=True)
    file.rename(Path(mask_directory, *file.parts[1:]))


Загрузить дата-сет

In [13]:
# dataset = keras.utils.image_dataset_from_directory('Whale ReId 2_mm', labels='inferred', shuffle='False', batch_size=64, interpolation='mitchellcubic',color_mode='rgb')
# masks = keras.utils.image_dataset_from_directory('Whale ReId 2_mm masked', labels='inferred', shuffle='False', batch_size=64, color_mode='grayscale')

train_ds, val_ds = keras.utils.image_dataset_from_directory('Whale ReId 2_mm', labels='inferred', validation_split=0.2, subset='both', seed=1337, batch_size=128, interpolation='mitchellcubic', color_mode='rgb', image_size=(256, 256))

Found 33644 files belonging to 102 classes.
Using 26916 files for training.
Using 6728 files for validation.


Модель (?)